In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

2025-09-30 14:57:33 __main__ INFO     torch.__version__='2.7.0+cu126', torch.version.cuda='12.6'
2025-09-30 14:57:33 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=8, torch.cuda.get_device_name()='NVIDIA A100 80GB PCIe'
2025-09-30 14:57:33 __main__ INFO     transformers.__version__='4.55.3'


In [3]:
from src.utils.training_utils import get_device_map

# model_key = "meta-llama/Llama-3.2-3B"
# model_key = "meta-llama/Llama-3.1-8B"
# model_key = "meta-llama/Llama-3.1-70B-Instruct"
# model_key = "meta-llama/Llama-3.3-70B-Instruct"
# model_key = "meta-llama/Llama-3.1-405B-Instruct"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-3-12b-it"
model_key = "google/gemma-2-27b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
# model_key = "Qwen/Qwen2.5-14B-Instruct"
# model_key = "Qwen/Qwen2.5-32B-Instruct"
# model_key = "Qwen/Qwen2.5-72B-Instruct"

# model_key = "Qwen/Qwen3-1.7B"
# model_key = "Qwen/Qwen3-4B"
# model_key = "Qwen/Qwen3-8B"
# model_key = "Qwen/Qwen3-14B"
# model_key = "Qwen/Qwen3-32B"

# device_map = get_device_map(model_key, 30, n_gpus=8)
# device_map

2025-09-30 14:57:37 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-09-30 14:57:37 git.cmd DEBUG    Popen(['git', 'version'], cwd=/disk/u/arnab/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-09-30 14:57:37 wandb.docker.auth DEBUG    Trying paths: ['/disk/u/arnab/.docker/config.json', '/disk/u/arnab/.dockercfg']
2025-09-30 14:57:37 wandb.docker.auth DEBUG    No config file found


In [4]:
from src.models import ModelandTokenizer

# from transformers import BitsAndBytesConfig

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # torch_dtype=torch.float32,
    # device_map=device_map,
    device_map="auto",
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
    attn_implementation="eager",
)

2025-09-30 14:57:38 src.models WARNING  google/gemma-2-27b-it not found in /disk/u/arnab/Codes/Models
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-09-30 14:57:38 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443
2025-09-30 14:57:38 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /google/gemma-2-27b-it/resolve/main/config.json HTTP/1.1" 200 0
2025-09-30 14:57:38 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /google/gemma-2-27b-it/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
2025-09-30 14:57:38 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/models/google/gemma-2-27b-it/tree/main/additional_chat_templates?recursive=False&expand=False HTTP/1.1" 404 64


Loading checkpoint shards:   0%|          | 0/12 [00:00<?, ?it/s]

2025-09-30 14:57:56 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /google/gemma-2-27b-it/resolve/main/generation_config.json HTTP/1.1" 200 0
2025-09-30 14:57:56 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /google/gemma-2-27b-it/resolve/main/custom_generate/generate.py HTTP/1.1" 404 0
2025-09-30 14:57:56 src.models INFO     loaded model <google/gemma-2-27b-it> | size: 51931.626 MB | dtype: torch.bfloat16 | device: cuda:0


In [5]:
from src.selection.data import SelectOneTask, SelectOrderTask

#################################################################################
# TASK_CLS = SelectOrderTask
# prompt_template_idx = 1
TASK_CLS = SelectOneTask
prompt_template_idx = 3
N_DISTRACTORS = 5
OPTION_STYLE = "single_line"
#################################################################################

select_task = TASK_CLS.load(
    path=os.path.join(
        env_utils.DEFAULT_DATA_DIR, 
        "selection", 
        # "profession.json"
        # "nationality.json"
        "objects.json"
    )
)

print(select_task)

['name', 'prompt_templates', 'odd_one_prompt_templates', 'order_prompt_templates', 'count_prompt_templates', 'yes_no_prompt_templates', 'first_item_in_cat_prompt_templates', 'last_item_in_cat_prompt_templates', 'categories', 'exclude_categories']
SelectOneTask: (different objects)
Categories: fruit(15), vehicle(15), furniture(15), animal(15), music instrument(15), clothing(15), electronics(14), sport equipment(15), kitchen appliance(15), vegetable(14), building(15), office supply(15), bathroom item(15), flower(15), tree(15), jewelry(15)



In [6]:
import copy
import random
from src.selection.utils import KeyedSet, get_first_token_id, verify_correct_option
from src.selection.data import SelectionSample
from src.functional import predict_next_token
from src.tokens import prepare_input
from src.selection.data import get_counterfactual_samples_within_task

In [7]:
patch_sample, clean_sample = get_counterfactual_samples_within_task(
    # patch_category="politician",
    # clean_category="actor",
    mt=mt,
    task=select_task,
    patch_category="fruit",
    clean_category="vehicle",
    filter_by_lm_prediction=True,
    prompt_template_idx=prompt_template_idx,
    option_style=OPTION_STYLE,
    distinct_options=True,
    patch_n_distractors=5,
    clean_n_distractors=5
)

# patch_sample.default_option_style = "single_line"
# clean_sample.default_option_style = "numbered"

print(patch_sample.prompt(), ">>", patch_sample.obj)
print(clean_sample.prompt(), ">>", clean_sample.obj)

2025-09-30 14:57:59 src.selection.data INFO     clean_obj_idx=4 | ['Pear', 'Coffee table', 'Football', 'Harmonica', 'Motorcycle', 'Rice cooker']
type(task)=<class 'src.selection.data.SelectOneTask'>
2025-09-30 14:58:00 src.selection.data INFO     Options: Van, Folder, Ring, Bookshelf, Dishwasher, Orange.
Which among these objects mentioned above is a fruit?
Answer:
2025-09-30 14:58:00 src.selection.data INFO     Watermelon | fruit -> Orange | pred=['" Orange"[20723] (p=0.930, logit=21.750)', '" **"[5231] (p=0.067, logit=19.125)', '"  "[139] (p=0.001, logit=14.812)', '"\n\n"[109] (p=0.001, logit=14.562)', '"Orange"[36565] (p=0.000, logit=13.438)']
2025-09-30 14:58:00 src.selection.data INFO     Options: Pear, Coffee table, Football, Harmonica, Motorcycle, Rice cooker.
Which among these objects mentioned above is a vehicle?
Answer:
2025-09-30 14:58:00 src.selection.data INFO     Car | vehicle -> Motorcycle | pred=['" Motorcycle"[82796] (p=0.984, logit=23.000)', '" **"[5231] (p=0.012, log

## Training Set

In [ ]:
from src.selection.data import CounterFactualSamplePair
from src.functional import free_gpu_cache
from src.selection.data import get_counterfactual_samples_interface
import random

train_samples_save_path = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR,
    "selection",
    "samples",
    "train",
    mt.name.split("/")[-1],
    select_task.task_name,
    "objects",
    # "profession",
    # "nationality",
    # "landmarks"
    # "rhymes"
)

os.makedirs(train_samples_save_path, exist_ok=True)


free_gpu_cache()
train_set = []
train_limit = 1024
start_from = 1

counterfactual_sampler = get_counterfactual_samples_interface[select_task.task_name]

while len(train_set) < train_limit:
    print(f"sample {len(train_set)+1} / {train_limit}")
    patch, clean = counterfactual_sampler(
        mt=mt,
        task=select_task,
        filter_by_lm_prediction=True,
        prompt_template_idx=3,
        option_style=OPTION_STYLE,
        n_distractors=random.choice(range(2, 6)),
    )
    train_set.append((clean, patch))
    cf_pair = CounterFactualSamplePair(
        patch_sample=patch,
        clean_sample=clean,
    )
    cf_pair.detensorize()
    with open(
        os.path.join(
            train_samples_save_path,
            f"{len(train_set) + start_from - 1:05d}.json",
        ),
        "w",
    ) as f:
        json.dump(cf_pair.to_dict(), f, indent=2)

len(train_set)

In [8]:
from src.selection.data import CounterFactualSamplePair
import random

train_set = []
train_limit = 1024

train_samples_load_path = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR,
    "selection",
    "samples",
    "train",
    mt.name.split("/")[-1],
    select_task.task_name,
    "objects",
    # "profession",
    # "nationality"
    # "landmarks",
    # "rhymes",
)

sample_files = [
    os.path.join(train_samples_load_path, f)
    for f in os.listdir(train_samples_load_path)
    if f.endswith(".json")
]
logger.info(f"Found {len(sample_files)} sample files")

prefix = ""
# prefix = "Recall the nationality of these people:\n"
# prefix = "Recall which country these landmarks are located in:\n"
# prefix = "Think about how these words sound when you say them aloud:\n"

random.shuffle(sample_files)
sample_files = sample_files[:train_limit]
for sample_file in sample_files:
    with open(sample_file, "r") as f:
        cf_pair_data = json.load(f)
    cf_pair = CounterFactualSamplePair.from_dict(cf_pair_data)
    # cf_pair.patch_sample.default_option_style = "bulleted"
    # cf_pair.clean_sample.default_option_style = "bulleted"

    patch_category = cf_pair.patch_sample.category
    random_category = random.choice(list(set(select_task.categories) - {patch_category}))
    random_obj = random.choice(select_task.category_wise_examples[random_category])
    cf_pair.patch_sample.options[cf_pair.patch_sample.obj_idx] = random_obj

    cf_pair.clean_sample.prompt_template = prefix + cf_pair.clean_sample.prompt_template
    cf_pair.patch_sample.prompt_template = prefix + cf_pair.patch_sample.prompt_template
    train_set.append((cf_pair.clean_sample, cf_pair.patch_sample))

len(train_set)

2025-09-30 14:58:08 __main__ INFO     Found 1024 sample files


1024

## Validation Set

In [ ]:
from src.selection.data import CounterFactualSamplePair
from src.functional import free_gpu_cache
from src.selection.data import get_counterfactual_samples_interface
import random

# data_type = "objects"
data_type = "profession"
# data_type = "nationality"
select_val_task = TASK_CLS.load(
    path=os.path.join(
        env_utils.DEFAULT_DATA_DIR, 
        "selection", 
        f"{data_type}.json"
    )
)

validation_samples_save_path = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR,
    "selection",
    "samples",
    "validation",
    mt.name.split("/")[-1],
    select_val_task.task_name,
    data_type,
)

os.makedirs(validation_samples_save_path, exist_ok=True)

    
free_gpu_cache()
validation_set = []
validation_limit = 512
start_from = 0

counterfactual_sampler = get_counterfactual_samples_interface[select_task.task_name]

while len(validation_set) < validation_limit:
    print(f"sample {len(validation_set)+1} / {validation_limit}")
    patch, clean = counterfactual_sampler(
        mt=mt,
        task=select_val_task,
        filter_by_lm_prediction=True,
        prompt_template_idx=3,
        option_style=OPTION_STYLE,
        n_distractors=random.choice(range(2, 6)),
    )
    validation_set.append((clean, patch))
    cf_pair = CounterFactualSamplePair(
        patch_sample=patch,
        clean_sample=clean,
    )
    cf_pair.detensorize()
    with open(
        os.path.join(
            validation_samples_save_path,
            f"{len(validation_set) + start_from - 1:05d}.json",
        ),
        "w",
    ) as f:
        json.dump(cf_pair.to_dict(), f, indent=2)

len(validation_set)

['name', 'prompt_templates', 'odd_one_prompt_templates', 'categories', 'exclude_categories']
sample 1 / 512
2025-09-30 18:03:16 src.selection.data INFO     clean_obj_idx=1 | ['Chris Hemsworth', 'Pedri', 'Kevin Durant', 'Mitch McConnell']
type(task)=<class 'src.selection.data.SelectOneTask'>
2025-09-30 18:03:16 src.selection.data INFO     Options: Russell Westbrook, Kylian Mbappé, Bernie Sanders, Chuck Todd.
Who among these people mentioned above is a politician by profession?
Answer:
2025-09-30 18:03:16 src.selection.data INFO     Gavin Newsom | politician -> Bernie Sanders | pred=['" Bernie"[80123] (p=0.938, logit=21.125)', '" **"[5231] (p=0.060, logit=18.375)', '"\n\n"[109] (p=0.001, logit=13.938)', '"  "[139] (p=0.001, logit=13.875)', '"\n"[108] (p=0.000, logit=12.812)']
2025-09-30 18:03:16 src.selection.data INFO     Options: Chris Hemsworth, Pedri, Kevin Durant, Mitch McConnell.
Who among these people mentioned above is a soccer player by profession?
Answer:
2025-09-30 18:03:16 sr

In [56]:
from src.selection.data import CounterFactualSamplePair
import random

# data_type = "profession"
data_type = "objects"

validation_set = []
validation_limit = 512

validation_samples_load_path = os.path.join(
    env_utils.DEFAULT_RESULTS_DIR,
    "selection",
    "samples",
    "validation",
    mt.name.split("/")[-1],
    select_task.task_name,
    data_type,
)

sample_files = [
    os.path.join(validation_samples_load_path, f)
    for f in os.listdir(validation_samples_load_path)
    if f.endswith(".json")
]
logger.info(f"Found {len(sample_files)} sample files")

prefix = ""
# prefix = "Recall the nationality of these people:\n"
# prefix = "Recall which country these landmarks are located in:\n"
# prefix = "Think about how these words sound when you say them aloud:\n"

random.shuffle(sample_files)
sample_files = sample_files[:validation_limit]
for sample_file in sample_files:
    with open(sample_file, "r") as f:
        cf_pair_data = json.load(f)
    cf_pair = CounterFactualSamplePair.from_dict(cf_pair_data)
    # cf_pair.patch_sample.default_option_style = "bulleted"
    # cf_pair.clean_sample.default_option_style = "bulleted"

    cf_pair.clean_sample.prompt_template = prefix + cf_pair.clean_sample.prompt_template
    cf_pair.patch_sample.prompt_template = prefix + cf_pair.patch_sample.prompt_template
    validation_set.append((cf_pair.clean_sample, cf_pair.patch_sample))

len(validation_set)

2025-09-30 17:26:12 __main__ INFO     Found 538 sample files


512

## DAS

In [12]:
from src.utils.typing import TokenizerOutput
from src.functional import (
    get_module_nnsight,
    patch_with_baukit,
    get_hs,
    interpret_logits,
)
from src.tokens import prepare_input
from itertools import product
from src.functional import PatchSpec
from src.selection.utils import get_first_token_id
import baukit


def patch_linear_subspaces(
    mt: ModelandTokenizer,
    base_input: TokenizerOutput,
    rotator: dict[str, torch.nn.Module],
    patch_locations: list[tuple[str, int]],
    source_hidden_states: dict[tuple[str, int], torch.Tensor],
    rotate_dimensions: int = 100,
    with_grad=False,
):
    module_to_tok_indices = {}
    for module_name, token_index in patch_locations:
        if module_name not in module_to_tok_indices:
            module_to_tok_indices[module_name] = []
        module_to_tok_indices[module_name].append(token_index)
    for module_name in module_to_tok_indices:
        if module_name not in rotator:
            raise ValueError(
                f"Rotator for module {module_name} not found in rotator dict"
            )
        if rotator[module_name] is None:
            raise ValueError(f"Rotator for module {module_name} is None")

    grad_env = torch.enable_grad if with_grad else torch.no_grad
    with grad_env():
        with mt.trace(base_input) as tracer:
            for module_name in module_to_tok_indices:
                module = get_module_nnsight(mt, module_name)

                # get the base representation
                base_state = (
                    module.output
                    if ("mlp" in module_name or module_name == mt.embedder_name)
                    else module.output[0]
                )
                rotator[module_name] = rotator[module_name].to(base_state.device)

                #! takes too long
                # try:
                #     device = rotator[module_name].weight.device
                #     dtype = rotator[module_name].weight.dtype
                #     rotator_inverse = torch.linalg.pinv(
                #         rotator[module_name].weight.float()
                #     ).to(device=device, dtype=dtype)
                # except:
                #     print("exception")
                #     rotator_inverse = rotator[module_name].weight.T

                #! becase the rotator should be kept orthogonal
                #! there will be some precision error with bfloat16
                rotator_inverse = rotator[module_name].weight.T 
                for token_idx in module_to_tok_indices[module_name]:
                    base_token_state = base_state[:, token_idx, :].clone()
                    # rotate the base representation
                    rotated_base = rotator[module_name](base_token_state)

                    # rotate the source representation
                    source_state = source_hidden_states[(module_name, token_idx)]
                    if source_state.ndim == 1:
                        source_state = source_state.unsqueeze(0)
                    rotated_source = rotator[module_name](source_state)
                    # print(f"{rotated_base.shape=}, {rotated_source.shape=}")

                    # patch the first `rotate_dimensions` dimensions of the source representation
                    rotated_patch = torch.cat(
                        [
                            rotated_source[:, :rotate_dimensions],
                            rotated_base[:, rotate_dimensions:],
                        ],
                        dim=1,
                    )

                    # inverse rotate the patched representation
                    patch = torch.matmul(rotated_patch, rotator_inverse.T)

                    # replace the base representation with the patched representation
                    base_state[:, token_idx, :] = patch

            output = mt.output.save()
    return output


# TESTING

## get a validation sample
source, destination = validation_set[11]
print(source.prompt(), ">>", source.obj)
print(destination.prompt(), ">>", destination.obj)

source_tokenized = prepare_input(prompts=[source.prompt()], tokenizer=mt)
destination_tokenized = prepare_input(prompts=[destination.prompt()], tokenizer=mt)

print("destination run")
destination_output = patch_with_baukit(mt=mt, inputs=destination_tokenized, patches=[])
destination_logits = destination_output.logits[:, -1, :]
destination_pred, destination_track = interpret_logits(
    logits=destination_logits,
    tokenizer=mt.tokenizer,
    interested_tokens=[
        get_first_token_id(opt, mt.tokenizer) for opt in destination.options
    ],
)
print(f"{[str(pred) for pred in destination_pred]}")
print(f"{[str(pred) for tok_id, (rank, pred) in destination_track.items()]}")


print("source run")
source_output = patch_with_baukit(mt=mt, inputs=source_tokenized, patches=[])
source_logits = source_output.logits[:, -1, :]
source_pred, source_track = interpret_logits(
    logits=source_logits,
    tokenizer=mt.tokenizer,
    interested_tokens=[get_first_token_id(opt, mt.tokenizer) for opt in source.options],
)
print(f"{[str(pred) for pred in source_pred]}")
print(f"{[str(pred) for tok_id, (rank, pred) in source_track.items()]}")

print("-" * 80)

## get patch hidden states
################################
layers = [mt.layer_name_format.format(25)]
token_indices = [-3, -2, -1]
################################
patch_locations = list(product(layers, token_indices))
source_hidden_states = get_hs(
    mt=mt, input=source_tokenized, locations=patch_locations, return_dict=True
)

## check the effect of patching the full residual states
patches = []
for module_name, token_index in patch_locations:
    patch_spec = PatchSpec(
        location=(module_name, token_index),
        patch=source_hidden_states[(module_name, token_index)],
        strategy="replace",
    )
    patches.append(patch_spec)

residual_patched_output = patch_with_baukit(
    mt=mt, inputs=destination_tokenized, patches=patches
)

print("residual patched run")
residual_patched_logits = residual_patched_output.logits[:, -1, :]
residual_patched_pred, residual_patched_track = interpret_logits(
    logits=residual_patched_logits,
    tokenizer=mt.tokenizer,
    interested_tokens=[
        get_first_token_id(opt, mt.tokenizer) for opt in destination.options
    ],
)
print(f"{[str(pred) for pred in residual_patched_pred]}")
print(f"{[str(pred) for tok_id, (rank, pred) in residual_patched_track.items()]}")

print("\n\nTESTING DAS")
rotator = {}
for layer_name in set([loc[0] for loc in patch_locations]):
    module = baukit.get_module(mt._model, layer_name)
    module_device = next(module.parameters()).device
    linear = torch.nn.Linear(mt.n_embd, mt.n_embd, bias=False).to(device=module_device)

    # initialize as identify
    # torch.nn.init.eye_(linear.weight)
    # initialize as orthogonal
    torch.nn.init.orthogonal_(linear.weight)

    rotator[layer_name] = linear.to(device=module_device, dtype=mt.dtype)

das_patched_output = patch_linear_subspaces(
    mt=mt,
    base_input=destination_tokenized,
    rotator=rotator,
    patch_locations=patch_locations,
    source_hidden_states=source_hidden_states,
    rotate_dimensions=mt.n_embd,
    with_grad=False,
)
das_patched_logits = das_patched_output.logits[:, -1, :]
das_patched_pred, das_patched_track = interpret_logits(
    logits=das_patched_logits,
    tokenizer=mt.tokenizer,
    interested_tokens=[
        get_first_token_id(opt, mt.tokenizer) for opt in destination.options
    ],
)
print(f"{[str(pred) for pred in das_patched_pred]}")
print(f"{[str(pred) for tok_id, (rank, pred) in das_patched_track.items()]}")

Options: Tiara, Juicer, Table.
Which among these objects mentioned above is a jewelry?
Answer: >> Tiara
Options: Anklet, Desk, Carnation.
Which among these objects mentioned above is a furniture?
Answer: >> Desk
destination run
['" Desk"[47035] (p=0.879, logit=18.625)', '" **"[5231] (p=0.093, logit=16.375)', '"\n\n"[109] (p=0.005, logit=13.438)', '"Desk"[63983] (p=0.004, logit=13.312)', '"  "[139] (p=0.004, logit=13.125)']
['" Desk"[47035] (p=0.879, logit=18.625)', '" Ank"[102549] (p=0.002, logit=12.250)', '" Car"[2244] (p=0.000, logit=7.562)']
source run
['" Ti"[7996] (p=0.828, logit=18.250)', '" **"[5231] (p=0.127, logit=16.375)', '" tiara"[151274] (p=0.013, logit=14.125)', '"\n\n"[109] (p=0.008, logit=13.625)', '"  "[139] (p=0.007, logit=13.438)']
['" Ti"[7996] (p=0.828, logit=18.250)', '" Table"[5707] (p=0.000, logit=7.094)', '" Ju"[4254] (p=0.000, logit=6.344)']
--------------------------------------------------------------------------------
residual patched run
['" Ank"[102549] (

In [ ]:
from torch.optim import AdamW
from src.selection.data import get_options_for_answer
from src.functional import free_gpu_cache
from src.utils.typing import PathLike


def get_optimal_rotation(
    mt: ModelandTokenizer,
    train_set: list[tuple[SelectionSample, SelectionSample]],
    layers: list[str] = [mt.layer_name_format.format(25)],
    token_indices: int = [-1],
    rotation_n_dim: int = 100,
    learning_rate: float = 1e-3,
    ortho_reg: float = 0.1,
    n_epochs: int = 5,
    batch_size: int = 4,
    save_path: PathLike | None = None,
    save_step: int = 5
):
    hparams = {
        "model": mt.name,
        "learning_rate": learning_rate,
        "n_epochs": n_epochs,
        "batch_size": batch_size,
        "layers": layers,
        "token_indices": token_indices,
    }
    logger.debug(f"Training with hparams: {hparams}")

    patch_locations = list(product(layers, token_indices))
    rotator = {}
    for layer_name in layers:
        module = baukit.get_module(mt._model, layer_name)
        module_device = next(module.parameters()).device
        linear = torch.nn.Linear(mt.n_embd, mt.n_embd, bias=False).to(
            device=module_device
        )
        # initialize as orthogonal
        torch.nn.init.orthogonal_(linear.weight)
        rotator[layer_name] = linear.to(device=module_device, dtype=mt.dtype)

    optimizer = AdamW(
        params=[linear.weight for linear in rotator.values()],
        lr=learning_rate,
        weight_decay=0.0,
    )

    batches = []
    for batch_start in range(0, len(train_set), batch_size):
        batches.append(train_set[batch_start : batch_start + batch_size])

    losses = []
    for epoch in range(n_epochs):
        epoch_loss = 0
        for batch_idx, batch in enumerate(batches):
            optimizer.zero_grad()

            batch_size_actual = len(batch)

            destination_samples, source_samples = zip(*batch)
            prompts = []
            prompts.extend([sample.prompt() for sample in destination_samples])
            prompts.extend([sample.prompt() for sample in source_samples])
            tokenized = prepare_input(prompts=prompts, tokenizer=mt)
            destination_tokenized = TokenizerOutput(
                data={k: v[: len(destination_samples), :] for k, v in tokenized.items()}
            )
            source_tokenized = TokenizerOutput(
                data={k: v[len(destination_samples) :, :] for k, v in tokenized.items()}
            )
            batch_target_tokens = [
                clean_sample.metadata["track_type_obj_token_id"]
                for clean_sample in destination_samples
            ]
            batch_distractor_tokens = [
                [
                    get_first_token_id(tokenizer=mt.tokenizer, name=opt, prefix=" ")
                    for opt in get_options_for_answer(destination_sample)
                    if get_first_token_id(tokenizer=mt.tokenizer, name=opt, prefix=" ")
                    != destination_sample.metadata["track_type_obj_token_id"]
                ]
                for destination_sample in destination_samples
            ]

            gold_prompts = []
            for source_sample, destination_sample in zip(
                source_samples, destination_samples
            ):
                gold_sample = copy.deepcopy(destination_sample)
                gold_sample.category = source_sample.category
                gold_prompts.append(gold_sample.prompt())
            gold_tokenized = prepare_input(prompts=gold_prompts, tokenizer=mt)

            debug = True
            if debug and batch_idx == 0 and epoch == 0:
                print("source prompt:", source_samples[0].prompt(), ">>", source_samples[0].obj)
                print("destination prompt:", destination_samples[0].prompt(), ">>", destination_samples[0].obj)
                print("gold prompt:", gold_prompts[0])

            source_hidden_states = get_hs(
                mt=mt,
                input=source_tokenized,
                locations=patch_locations,
                return_dict=True,
            )

            das_patched_output = patch_linear_subspaces(
                mt=mt,
                base_input=destination_tokenized,
                rotator=rotator,
                patch_locations=patch_locations,
                source_hidden_states=source_hidden_states,
                rotate_dimensions=rotation_n_dim,
                with_grad=True,
            )
            logits = das_patched_output.logits[:, -1, :]

            # # calculate target loss
            # target_logits = [
            #     logit[tok] for logit, tok in zip(logits, batch_target_tokens)
            # ]
            # target_loss = -torch.stack(target_logits).mean()  # need this to go up

            # # calculate distractor loss
            # distractor_logits = [
            #     logit[distractor_tokens].mean()
            #     for logit, distractor_tokens in zip(logits, batch_distractor_tokens)
            # ]
            # distractor_loss = torch.stack(distractor_logits).mean()

            # calculate target loss
            with torch.no_grad():
                gold_output = patch_with_baukit(
                    mt=mt, inputs=gold_tokenized, patches=[]
                )
                gold_logits = gold_output.logits[:, -1, :]

            kldiv_loss = torch.nn.functional.kl_div(
                input=logits.log_softmax(dim=-1),
                target=gold_logits.softmax(dim=-1),
                reduction="batchmean",
            )

            # rotator orthogonality loss
            orthogonality_losses = []
            for layer_name in layers:
                identity = torch.eye(mt.n_embd).to(rotator[layer_name].weight.device)
                wt_w = torch.matmul(
                    rotator[layer_name].weight.T, rotator[layer_name].weight
                )
                orthogonality_loss = ((wt_w - identity) ** 2).mean()
                orthogonality_losses.append(orthogonality_loss)
            # ortho_loss = torch.stack(orthogonality_losses).mean().to(target_loss.device)
            ortho_loss = torch.stack(orthogonality_losses).mean().to(kldiv_loss.device)


            # total loss
            # loss = target_loss + distractor_loss + ortho_reg * ortho_loss
            # logger.debug(
            #     f"Epoch={epoch+1} | {batch_idx=} |>> {target_loss.item():.4f} + {distractor_loss.item():.4f} + {ortho_loss.item():.4f} = {loss.item():.4f}"
            # )

            loss = kldiv_loss + ortho_reg * ortho_loss
            logger.debug(
                f"Epoch={epoch+1} | {batch_idx=} |>> {kldiv_loss.item():.4f} + {ortho_loss.item():.4f} = {loss.item():.4f}"
            )

            loss.backward()
            optimizer.step()

            epoch_loss += loss.item() * batch_size_actual
            losses.append(loss.item())

        epoch_loss = epoch_loss / len(train_set)
        logger.info(f"Epoch {epoch+1} completed. Avg Loss: {epoch_loss:.4f}")
        mt._model.zero_grad()
        free_gpu_cache()

        # TODO: save intermediate rotator
        if save_path is not None and ((epoch + 1) % save_step == 0 or (epoch + 1) == n_epochs):
            os.makedirs(save_path, exist_ok=True)
            rotator_save_path = os.path.join(
                save_path, f"rotator_epoch{epoch+1:03d}.pt"
            )
            torch.save(rotator, rotator_save_path)
            logger.info(f"Saved rotator at {rotator_save_path}")

    mt._model.zero_grad()
    for module_name, proj in rotator.items():
        rotator[module_name].requires_grad_(False)

    free_gpu_cache()
    return rotator, losses


mt._model.zero_grad()
free_gpu_cache()

optimal_rotator, training_losses = get_optimal_rotation(
    mt=mt,
    train_set=train_set,
    layers=[mt.layer_name_format.format(25)],
    token_indices=[-3, -2, -1],
    rotation_n_dim=256,
    learning_rate=1e-3,
    ortho_reg=0.1,
    n_epochs=10,
    batch_size=32,
)

2025-09-30 17:00:31 __main__ DEBUG    Training with hparams: {'model': 'google/gemma-2-27b-it', 'learning_rate': 0.001, 'n_epochs': 10, 'batch_size': 32, 'layers': ['model.layers.25'], 'token_indices': [-3, -2, -1]}
2025-09-30 17:00:35 __main__ DEBUG    Epoch=1 | batch_idx=0 |>> 8.0625 + 0.0000 = 8.0625
2025-09-30 17:00:36 __main__ DEBUG    Epoch=1 | batch_idx=1 |>> 23.1250 + 0.0000 = 23.1250
2025-09-30 17:00:39 __main__ DEBUG    Epoch=1 | batch_idx=2 |>> 9.1875 + 0.0000 = 9.1875
2025-09-30 17:00:41 __main__ DEBUG    Epoch=1 | batch_idx=3 |>> 8.7500 + 0.0001 = 8.7500
2025-09-30 17:00:43 __main__ DEBUG    Epoch=1 | batch_idx=4 |>> 7.7812 + 0.0003 = 7.7813
2025-09-30 17:00:44 __main__ DEBUG    Epoch=1 | batch_idx=5 |>> 7.8125 + 0.0005 = 7.8126
2025-09-30 17:00:46 __main__ DEBUG    Epoch=1 | batch_idx=6 |>> 7.5938 + 0.0008 = 7.5938
2025-09-30 17:00:48 __main__ DEBUG    Epoch=1 | batch_idx=7 |>> 7.8750 + 0.0011 = 7.8751
2025-09-30 17:00:49 __main__ DEBUG    Epoch=1 | batch_idx=8 |>> 7.2188

In [67]:
# TESTING

## get a validation sample
destination, source = validation_set[5]
# destination, source = train_set[7]
print(source.prompt(), ">>", source.obj)
print(destination.prompt(), ">>", destination.obj)

source_tokenized = prepare_input(prompts=[source.prompt()], tokenizer=mt)
destination_tokenized = prepare_input(prompts=[destination.prompt()], tokenizer=mt)

print("destination run")
destination_output = patch_with_baukit(mt=mt, inputs=destination_tokenized, patches=[])
destination_logits = destination_output.logits[:, -1, :]
destination_pred, destination_track = interpret_logits(
    logits=destination_logits,
    tokenizer=mt.tokenizer,
    interested_tokens=[
        get_first_token_id(opt, mt.tokenizer) for opt in destination.options
    ],
)
print(f"{[str(pred) for pred in destination_pred]}")
print(f"{[str(pred) for tok_id, (rank, pred) in destination_track.items()]}")


print("source run")
source_output = patch_with_baukit(mt=mt, inputs=source_tokenized, patches=[])
source_logits = source_output.logits[:, -1, :]
source_pred, source_track = interpret_logits(
    logits=source_logits,
    tokenizer=mt.tokenizer,
    interested_tokens=[get_first_token_id(opt, mt.tokenizer) for opt in source.options],
)
print(f"{[str(pred) for pred in source_pred]}")
print(f"{[str(pred) for tok_id, (rank, pred) in source_track.items()]}")

print("=" * 80)
print(destination.metadata["track_type_obj"])
gold_sample = copy.deepcopy(destination)
gold_sample.category = source.category
print(gold_sample.prompt())
gold_tokenized = prepare_input(prompts=[gold_sample.prompt()], tokenizer=mt)
gold_output = patch_with_baukit(mt=mt, inputs=gold_tokenized, patches=[])
gold_logits = gold_output.logits[:, -1, :]
gold_pred, gold_track = interpret_logits(
    logits=gold_logits,
    tokenizer=mt.tokenizer,
    interested_tokens=[
        get_first_token_id(opt, mt.tokenizer) for opt in destination.options
    ],
)
print(f"{[str(pred) for pred in gold_pred]}")
print(f"{[str(pred) for tok_id, (rank, pred) in gold_track.items()]}")
print("=" * 80)

## get patch hidden states
################################
layers = [mt.layer_name_format.format(25)]
token_indices = [-3, -2, -1]
################################
patch_locations = list(product(layers, token_indices))
source_hidden_states = get_hs(
    mt=mt, input=source_tokenized, locations=patch_locations, return_dict=True
)

## check the effect of patching the full residual states
patches = []
for module_name, token_index in patch_locations:
    patch_spec = PatchSpec(
        location=(module_name, token_index),
        patch=source_hidden_states[(module_name, token_index)],
        strategy="replace",
    )
    patches.append(patch_spec)

residual_patched_output = patch_with_baukit(
    mt=mt, inputs=destination_tokenized, patches=patches
)

print("residual patched run")
residual_patched_logits = residual_patched_output.logits[:, -1, :]
residual_patched_pred, residual_patched_track = interpret_logits(
    logits=residual_patched_logits,
    tokenizer=mt.tokenizer,
    interested_tokens=[
        get_first_token_id(opt, mt.tokenizer) for opt in destination.options
    ],
)
print(f"{[str(pred) for pred in residual_patched_pred]}")
print(f"{[str(pred) for tok_id, (rank, pred) in residual_patched_track.items()]}")

print("\n\nTESTING DAS")
rotator = {}
for layer_name in set([loc[0] for loc in patch_locations]):
    module = baukit.get_module(mt._model, layer_name)
    module_device = next(module.parameters()).device
    linear = torch.nn.Linear(mt.n_embd, mt.n_embd, bias=False).to(device=module_device)

    # initialize as identify
    # torch.nn.init.eye_(linear.weight)
    # initialize as orthogonal
    torch.nn.init.orthogonal_(linear.weight)

    rotator[layer_name] = linear.to(device=module_device, dtype=mt.dtype)

# das_patched_output = patch_linear_subspaces(
#     mt=mt,
#     base_input=destination_tokenized,
#     rotator=rotator,
#     patch_locations=patch_locations,
#     source_hidden_states=source_hidden_states,
#     rotate_dimensions=mt.n_embd,
#     with_grad=False,
# )

das_patched_output = patch_linear_subspaces(
    mt=mt,
    base_input=destination_tokenized,
    rotator=optimal_rotator,
    patch_locations=patch_locations,
    source_hidden_states=source_hidden_states,
    rotate_dimensions=256,
    with_grad=False,
)

das_patched_logits = das_patched_output.logits[:, -1, :]
das_patched_pred, das_patched_track = interpret_logits(
    logits=das_patched_logits,
    tokenizer=mt.tokenizer,
    interested_tokens=[
        get_first_token_id(opt, mt.tokenizer) for opt in destination.options
    ],
)
print(f"{[str(pred) for pred in das_patched_pred]}")
print(f"{[str(pred) for tok_id, (rank, pred) in das_patched_track.items()]}")

Options: Trumpet, Slow cooker, Watch, Scarf.
Which among these objects mentioned above is a kitchen appliance?
Answer: >> Slow cooker
Options: Oven, Potato, Cherry, Harmonica.
Which among these objects mentioned above is a music instrument?
Answer: >> Harmonica
destination run
['" Harmon"[60970] (p=0.809, logit=18.125)', '" **"[5231] (p=0.159, logit=16.500)', '"  "[139] (p=0.007, logit=13.438)', '"\n\n"[109] (p=0.007, logit=13.438)', '"\n"[108] (p=0.002, logit=12.125)']
['" Harmon"[60970] (p=0.809, logit=18.125)', '" Cherry"[44507] (p=0.000, logit=9.062)', '" Oven"[91129] (p=0.000, logit=6.812)', '" Potato"[67031] (p=0.000, logit=6.750)']
source run
['" Slow"[33936] (p=0.863, logit=18.250)', '" **"[5231] (p=0.103, logit=16.125)', '"  "[139] (p=0.007, logit=13.438)', '"\n\n"[109] (p=0.007, logit=13.438)', '" slow"[6080] (p=0.004, logit=12.812)']
['" Slow"[33936] (p=0.863, logit=18.250)', '" Trumpet"[158341] (p=0.001, logit=11.438)', '" Watch"[13466] (p=0.000, logit=9.625)', '" Scarf"[17

In [85]:
from tqdm.auto import tqdm

validation_results = []
for destination, source in tqdm(validation_set):
    destination = copy.deepcopy(destination)
    source = copy.deepcopy(source)
    destination.default_option_style = "bulleted"
    source_tokenized = prepare_input(prompts=[source.prompt()], tokenizer=mt)
    destination_tokenized = prepare_input(prompts=[destination.prompt()], tokenizer=mt)

    track_tokens = {    
        "clean_obj": destination.ans_token_id,
        "target_obj": destination.metadata["track_type_obj_token_id"],
    }

    source_hidden_states = get_hs(
        mt=mt,
        input=source_tokenized,
        locations=patch_locations,
        return_dict=True,
    )

    destination_output = patch_with_baukit(
        mt=mt, inputs=destination_tokenized, patches=[]
    )
    destination_logits = destination_output.logits[:, -1, :]
    destination_pred, destination_track = interpret_logits(
        logits=destination_logits,
        tokenizer=mt.tokenizer,
        interested_tokens=[
            get_first_token_id(opt, mt.tokenizer) for opt in destination.options
        ],
    )

    das_patched_output = patch_linear_subspaces(
        mt=mt,
        base_input=destination_tokenized,
        rotator=optimal_rotator,
        patch_locations=patch_locations,
        source_hidden_states=source_hidden_states,
        rotate_dimensions=256,
        with_grad=False,
    )
    das_patched_logits = das_patched_output.logits[:, -1, :]
    das_patched_pred, das_patched_track = interpret_logits(
        logits=das_patched_logits,
        tokenizer=mt.tokenizer,
        interested_tokens=[
            get_first_token_id(opt, mt.tokenizer) for opt in destination.options
        ],
    )

    validation_results.append(
        {
            "destination": destination,
            "source": source,
            "patched_pred": das_patched_pred,
            "patched_track": das_patched_track,
            "destination_pred": destination_pred,
            "destination_track": destination_track,
            "track_tokens": track_tokens,
        }
    )

  0%|          | 0/512 [00:00<?, ?it/s]

In [87]:
import numpy as np

track_token_types = ["clean_obj", "target_obj"]

for token_type in track_token_types:
    print(f"{token_type}")
    ranks = {"clean": [], "patch": []}
    logits = {"clean": [], "patch": []}
    for result in validation_results:
        target_tok_id = result["track_tokens"][token_type]
        clean_rank, clean_pred = result["destination_track"][target_tok_id]
        patch_rank, patch_pred = result["patched_track"][target_tok_id]
        ranks["clean"].append(clean_rank)
        ranks["patch"].append(patch_rank)
        logits["clean"].append(clean_pred.logit)
        logits["patch"].append(patch_pred.logit)

    attr = {
        "rank": ranks,
        "logit": logits,
    }
    for key in attr:
        clean = np.array(attr[key]["clean"])
        patch = np.array(attr[key]["patch"])
        delta = patch - clean
        print(f"{key}: clean {clean.mean():.2f} ± {clean.std():.2f} -> patch {patch.mean():.2f} ± {patch.std():.2f} | delta {delta.mean():.2f} ± {delta.std():.2f}")


top_1_accuracy = 0
for result in validation_results:
    target_tok_id = result["destination"].metadata["track_type_obj_token_id"]
    patched_track = result["patched_track"]
    if (
        patched_track[list(patched_track.keys())[0]][1].token_id == target_tok_id
    ):
        top_1_accuracy += 1

top_1_accuracy = top_1_accuracy / len(validation_results)
print("top 1 accuracy:", top_1_accuracy)
    

clean_obj
rank: clean 1.12 ± 0.67 -> patch 9670.26 ± 27534.83 | delta 9669.14 ± 27534.84
logit: clean 18.15 ± 0.64 -> patch 13.43 ± 7.03 | delta -4.72 ± 7.09
target_obj
rank: clean 377.50 ± 845.82 -> patch 1.76 ± 2.15 | delta -375.75 ± 845.39
logit: clean 7.20 ± 2.47 -> patch 25.51 ± 1.76 | delta 18.31 ± 2.81
top 1 accuracy: 0.849609375


In [88]:
select_task.prompt_templates

['Which object from the following list shares its category with <_pivot_entity_>?\n<_options_>\nAnswer:',
 '<_options_>\nWhich among these objects mentioned above share the same category as <_pivot_entity_>?\nAnswer:',
 'Which object from the following list is a <_category_>?\n<_options_>\nAnswer:',
 '<_options_>\nWhich among these objects mentioned above is a <_category_>?\nAnswer:']